In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *
import pandas as pd
#from sens import get_dsdp
from pyomo.contrib.sensitivity_toolbox.sens import get_dsdp, get_dfds_dcds
#from idaes.apps.uncertainty_propagation.uncertainties import propagate_uncertainty
#from idaes.apps.uncertainty_propagation.sens import get_dsdp
import time

In [2]:
time_an = list(np.linspace(0,1,9))

### Analytical model, without parameter index

In [3]:

def create_model2(CA_init=5, T_init=300):
    '''Model for get_sensitivity, defining measurements as cosntraints
    '''
    
    # time points 
    #timepoint = [0.0, 0.125, 0.25, 0.375, 0.5,0.625, 0.75, 0.875, 1.0]
    timepoint = time_an
    # names of observed variables
    y_list = ['CA','CB','CC']
    theta_p = {'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}
    
    m= ConcreteModel()

    m.CA0 = CA_init
    m.T = T_init
    
    # Define ideal gas constant
    m.R = 8.31446261815324 # J / K / mol

    ### Define sets and expressions
    # timepoint
    m.t = Set(initialize=timepoint)
    
    # response/observed variables
    m.y = Set(initialize=y_list)

    m.A1 = Var(initialize = theta_p['A1'])
    m.A2 = Var(initialize = theta_p['A2'])
    m.E1 = Var(initialize = theta_p['E1'])
    m.E2 = Var(initialize = theta_p['E2'])
    
    # Concentration of A, B, and C [mol/L]
    m.C = Var(m.y, m.t, initialize=0.0, within=NonNegativeReals)
    m.dCdt = Var(m.y, m.t, initialize=0.0, within=NonNegativeReals)

    def k1_rule(m):
        return m.A1*exp(-m.E1*1000/m.R/m.T)

    def k2_rule(m):
        return m.A2 * exp(-m.E2*1000/m.R/m.T)

    m.k1 = Expression(rule = k1_rule) # 1/hr
    m.k2 = Expression(rule = k2_rule)
    
    def conc(m,i,t):
        '''
        Calculate the model predictions
        Argument: 
            i: the model responses, CA, CB, CC
            t: timepoints
        '''
        if i == 'CA':
            return m.C['CA',t] == m.CA0*exp(-m.k1*t)
        elif i == 'CB':
            return m.C['CB',t] == m.k1*m.CA0/(m.k2-m.k1) * (exp(-m.k1*t) - exp(-m.k2*t))
        else:
            return m.C['CC',t] == m.CA0 - m.CA0*exp(-m.k1*t) - m.k1*m.CA0/(m.k2-m.k1) * (exp(-m.k1*t) - exp(-m.k2*t))

    m.rate = Constraint(m.y, m.t, rule=conc)
    
    def gradient(m, i, t):
        if i=='CA':
            return m.dCdt['CA',t] == -m.k1*m.C['CA',t]
        elif i=='CB':
            return m.dCdt['CB',t] == m.k1*m.C['CA',t] - m.k2*m.C['CB',t]
        elif i=='CC':
            return m.dCdt['CC',t] == m.k2*m.C['CB',t]
        
    #m.grad = Constraint(m.y, m.t, rule=gradient)

    def obj_rule(m):
        return 0
    
    m.Obj = Objective(rule=obj_rule, sense=maximize)
    
    for v in variable_name:
        getattr(m, v).setlb(theta_p[v])
        getattr(m, v).setub(theta_p[v])
        
    return m

In [4]:
def create_model(scena, CA_init=5, T_init=300, C_init=1):
    '''
    This is an example user model provided to DoE library. 
    It is a problem using algebraic equations.
    
    Arguments:
        scena: a dictionary of scenarios, achieved from scenario_generator()
    
        Time-independent design variable: 
            - CA_init: CA0 value
            - T_init: T value

        C_init: initial value for C
        
    Return:
        m: a Pyomo.DAE model 
    '''
    # parameters initialization, results from parameter estimation
    theta_pe = {'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}
    # concentration initialization
    y_init = {'CA': value(CA_init), 'CB':0.0, 'CC':0.0}
    
    para_list = ['A1', 'A2', 'E1', 'E2']
    
    ### Add variables 
    m = ConcreteModel()
    
    m.CA_init = CA_init
    m.para_list = para_list
    
    m.scena_all = scena 
    m.scena = Set(initialize=scena['scena-name'])
    
    # timepoints
    
    #m.t = Set(initialize=[0,0.125,0.25,0.375,0.5,0.625,0.75,0.875,1])
    m.t = Set(initialize=time_an)
    
    m.t_con = Set(initialize=[0])
    
    # time-independent design variable
    m.CA0 = Var(m.t_con, initialize = CA_init, bounds=(1.0,5.0), within=NonNegativeReals) # mol/L
    
    m.T = Var(m.t, initialize = T_init, bounds=(300, 700), within=NonNegativeReals)
     
    m.R = 8.31446261815324 # J / K / mole
       
    # Define parameters
    #m.A1 = Param(m.scena, initialize=m.scena_all['A1'],mutable=True)
    #m.A2 = Param(m.scena, initialize=m.scena_all['A2'],mutable=True)
    #m.E1 = Param(m.scena, initialize=m.scena_all['E1'],mutable=True)
    #m.E2 = Param(m.scena, initialize=m.scena_all['E2'],mutable=True)
    
    m.A1 = Var(m.scena, initialize = m.scena_all['A1'])
    m.A2 = Var(m.scena, initialize = m.scena_all['A2'])
    m.E1 = Var(m.scena, initialize = m.scena_all['E1'])
    m.E2 = Var(m.scena, initialize = m.scena_all['E2'])
    
    # Concentration variables under perturbation
    m.CA = Var(m.scena, m.t, initialize=C_init, within=NonNegativeReals)
    m.CB = Var(m.scena, m.t, initialize=C_init, within=NonNegativeReals)
    m.CC = Var(m.scena, m.t, initialize=C_init, within=NonNegativeReals)

    
    # kinetic parameters
    def kp1_init(m,s,t):
        return m.A1[s] * exp(-m.E1[s]*1000/(m.R*m.T[t]))
    
    def kp2_init(m,s,t):
        return m.A2[s] * exp(-m.E2[s]*1000/(m.R*m.T[t]))
    
    m.kp1 = Var(m.scena, m.t, initialize=kp1_init)
    m.kp2 = Var(m.scena, m.t, initialize=kp2_init)
        
    def cal_kp1(m,z,t):
        '''
        Create the perturbation parameter sets 
        m: model
        z: scenario number
        t: time
        '''
        # LHS: 1/h
        # RHS: 1/h*(kJ/mol *1000J/kJ / (J/mol/K) / K)
        return m.kp1[z,t] == m.A1[z]*exp(-m.E1[z]*1000/(m.R*m.T[t])) 
            
    def cal_kp2(m,z,t):
        '''
        Create the perturbation parameter sets 
        m: model
        z: m.pert, upper or normal or lower perturbation
        t: time
        '''
        # LHS: 1/h
        # RHS: 1/h*(kJ/mol *1000J/kJ / (J/mol/K) / K)
        return m.kp2[z,t] == m.A2[z]*exp(-m.E2[z]*1000/(m.R*m.T[t])) 
        
    # Calculate model response variables
    def CA_conc(m,z,t):
        '''
        Calculate the model predictions
        Argument: 
            z: scenario
            t: timepoints
        '''
        return m.CA[z,t] == m.CA0[0]*exp(-m.kp1[z,t]*t)
    
    def CB_conc(m,z,t):
        '''
        Calculate the model predictions
        Argument: 
            z: scenario
            t: timepoints
        '''
        return m.CB[z,t] == m.kp1[z,t]*m.CA0[0]/(m.kp2[z,t]-m.kp1[z,t]) * (exp(-m.kp1[z,t]*t) - exp(-m.kp2[z,t]*t))
    
    def CC_conc(m,z,t):
        '''
        Calculate the model predictions
        Argument: 
            z: scenario
            t: timepoints
        '''
        return m.CC[z,t] == m.CA0[0] - m.CA[z,t] - m.CB[z,t]
    
        
    def T_const(m,t):
        if t==0:
            return Constraint.Skip
        else:
            return m.T[t] == m.T[0]
        
    def obj_rule(m):
        return 0
    
    m.Obj = Objective(rule=obj_rule, sense=maximize)
        
        
    # calculating C, Jacobian, FIM
    m.k1_pert_rule = Constraint(m.scena, m.t, rule=cal_kp1)
    m.k2_pert_rule = Constraint(m.scena, m.t, rule=cal_kp2)
    m.dCAdt_rule = Constraint(m.scena, m.t, rule=CA_conc)
    m.dCBdt_rule = Constraint(m.scena, m.t, rule=CB_conc)
    m.dCCdt_rule = Constraint(m.scena, m.t, rule=CC_conc)
    #m.CA0_keep = Constraint(m.t, rule=CA0_const)
    #m.T_keep = Constraint(m.t, rule=T_const)
    
    
    return m 


In [5]:
from pyomo.environ import (
    Param, Var, Block, ComponentMap, Objective, Constraint,
    ConstraintList, Suffix, value, ComponentUID,
)


mod= create_model(scena={'scena-name': [0], 'A1':84.79085853498033,'A2': 371.71773413976416,
                             'E1':7.777032028026428, 'E2':15.047135137500822})


def add_comp(m):
    para_name = ['A1','A2','E1','E2']

    comp = m.find_component('A1')
    print(comp)

    #m.del_component('A1')
    #m.del_component('A2')
    #m.del_component('E1')
    #m.del_component('E2')

    m.CA0.fix()
    m.T.fix()
    #comp2 = m.find_component('A1')
    #print(comp2)

    for item in para_name:
        #setattr(m, item, Var(m.scena, initialize=m.scena_all[item]))
        compe = eval('m.' + item + '[0]')
        compe.setlb(m.scena_all[item])
        compe.setub(m.scena_all[item])

    print(m.A1)
    return m 

mod = add_comp(mod)
print(value(mod.A1[0]))

A1
A1
84.79085853498033


### Analytical model with scenario index

In [6]:
def create_model_scena(scena, CA_init=5, T_init=300, C_init=1):
    '''Model for get_sensitivity, defining measurements as cosntraints
    '''
    
    # parameters initialization, results from parameter estimation
    theta_p = {'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}
    # concentration initialization
    y_init = {'CA': value(CA_init), 'CB':0.0, 'CC':0.0}
    
    para_list = ['A1', 'A2', 'E1', 'E2']
    
    # names of observed variables
    y_list = ['CA','CB','CC']
    
    
    ### add variable 
    m= ConcreteModel()

    m.CA_init = CA_init
    m.para_list = para_list
    
    m.scena_all = scena
    m.scena = Set(initialize=scena['scena-name'])

    ### Define sets and expressions
    # timepoint
    m.t = Set(initialize=time_an)
    
    m.t_con = Set(initialize=[0])
    
    # time-independent design variable
    m.CA0 = Var(m.t_con, initialize = CA_init, bounds=(1.0,5.0), within=NonNegativeReals) # mol/L
    m.CA0.fix()
    
    m.T = Var(m.t, initialize = T_init, bounds=(300, 700), within=NonNegativeReals)
    m.T.fix()
     
    # Define ideal gas constant
    m.R = 8.31446261815324 # J / K / mol

    m.A1 = Var(m.scena, initialize = theta_p['A1'])
    m.A2 = Var(m.scena, initialize = theta_p['A2'])
    m.E1 = Var(m.scena, initialize = theta_p['E1'])
    m.E2 = Var(m.scena, initialize = theta_p['E2'])
    
    # Concentration of A, B, and C [mol/L]
    m.CA = Var(m.scena, m.t, initialize=0.0, within=NonNegativeReals)
    m.CB = Var(m.scena, m.t, initialize=0.0, within=NonNegativeReals)
    m.CC = Var(m.scena, m.t, initialize=0.0, within=NonNegativeReals)
    #m.dCdt = Var(m.y, m.t, initialize=0.0, within=NonNegativeReals)

    def k1_rule(m,s,t):
        return m.A1[s]*exp(-m.E1[s]*1000/m.R/m.T[t])

    def k2_rule(m,s,t):
        return m.A2[s] * exp(-m.E2[s]*1000/m.R/m.T[t])

    m.kp1 = Var(m.scena, m.t, initialize = k1_rule) # 1/hr
    m.kp2 = Var(m.scena, m.t, initialize = k2_rule)
    
    def cal_kp1(m,z,t):
        '''
        Create the perturbation parameter sets 
        m: model
        z: scenario number
        t: time
        '''
        # LHS: 1/h
        # RHS: 1/h*(kJ/mol *1000J/kJ / (J/mol/K) / K)
        return m.kp1[z,t] == m.A1[z]*exp(-m.E1[z]*1000/(m.R*m.T[t])) 
            
    def cal_kp2(m,z,t):
        '''
        Create the perturbation parameter sets 
        m: model
        z: m.pert, upper or normal or lower perturbation
        t: time
        '''
        # LHS: 1/h
        # RHS: 1/h*(kJ/mol *1000J/kJ / (J/mol/K) / K)
        return m.kp2[z,t] == m.A2[z]*exp(-m.E2[z]*1000/(m.R*m.T[t])) 
    
    # Calculate model response variables
    def CA_conc(m,z,t):
        '''
        Calculate the model predictions
        Argument: 
            z: scenario
            t: timepoints
        '''
        return m.CA[z,t] == m.CA0[0]*exp(-m.kp1[z,t]*t)
    
    def CB_conc(m,z,t):
        '''
        Calculate the model predictions
        Argument: 
            z: scenario
            t: timepoints
        '''
        return m.CB[z,t] == m.kp1[z,t]*m.CA0[0]/(m.kp2[z,t]-m.kp1[z,t]) * (exp(-m.kp1[z,t]*t) - exp(-m.kp2[z,t]*t))
    
    def CC_conc(m,z,t):
        '''
        Calculate the model predictions
        Argument: 
            z: scenario
            t: timepoints
        '''
        return m.CC[z,t] == m.CA0[0] - m.CA[z,t] - m.CB[z,t]
    
        
    def T_const(m,t):
        if t==0:
            return Constraint.Skip
        else:
            return m.T[t] == m.T[0]
        
        
    # calculating C, Jacobian, FIM
    m.k1_pert_rule = Constraint(m.scena, m.t, rule=cal_kp1)
    m.k2_pert_rule = Constraint(m.scena, m.t, rule=cal_kp2)
    m.dCAdt_rule = Constraint(m.scena, m.t, rule=CA_conc)
    m.dCBdt_rule = Constraint(m.scena, m.t, rule=CB_conc)
    m.dCCdt_rule = Constraint(m.scena, m.t, rule=CC_conc)
    #m.CA0_keep = Constraint(m.t, rule=CA0_const)
    #m.T_keep = Constraint(m.t, rule=T_const)
    

    def obj_rule(m):
        return 0
    
    m.Obj = Objective(rule=obj_rule, sense=maximize)
    
    for v in variable_name:
        m.A1[0].setlb(theta_p['A1'])
        m.A2[0].setlb(theta_p['A2'])
        m.E1[0].setlb(theta_p['E1'])
        m.E2[0].setlb(theta_p['E2'])
        m.A1[0].setub(theta_p['A1'])
        m.A2[0].setub(theta_p['A2'])
        m.E1[0].setub(theta_p['E1'])
        m.E2[0].setub(theta_p['E2'])
        
    return m

### DAE model 

In [7]:
def create_model_dae(scena, CA_init=5, T_init=300, C_init=1):
    '''Model for get_sensitivity, defining measurements as cosntraints
    '''
    
    # parameters initialization, results from parameter estimation
    theta_p = {'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}
    # concentration initialization
    y_init = {'CA': value(CA_init), 'CB':0.0, 'CC':0.0}
    
    para_list = ['A1', 'A2', 'E1', 'E2']
    
    # names of observed variables
    y_list = ['CA','CB','CC']
    
    t_control = time_an
    
    
    ### add variable 
    m= ConcreteModel()

    m.CA_init = CA_init
    m.para_list = para_list
    
    m.scena_all = scena
    m.scena = Set(initialize=scena['scena-name'])

    ### Define sets and expressions
    # timepoint
    m.t = ContinuousSet(bounds=(0.0,1))
    
    m.t_con = Set(initialize=t_control)
    
    m.t0 = Set(initialize=[0])
    
    # time-independent design variable
    m.CA0 = Var(m.t0, initialize = CA_init, bounds=(1.0,5.0), within=NonNegativeReals) # mol/L
    m.CA0.fix()
    
    m.T = Var(m.t, initialize = T_init, bounds=(300, 700), within=NonNegativeReals)

    # Define ideal gas constant
    m.R = 8.31446261815324 # J / K / mol

    m.A1 = Var(m.scena, initialize = theta_p['A1'])
    m.A2 = Var(m.scena, initialize = theta_p['A2'])
    m.E1 = Var(m.scena, initialize = theta_p['E1'])
    m.E2 = Var(m.scena, initialize = theta_p['E2'])
    
    # Concentration of A, B, and C [mol/L]
    m.CA = Var(m.scena, m.t, initialize=0.0, within=NonNegativeReals)
    m.CB = Var(m.scena, m.t, initialize=0.0, within=NonNegativeReals)
    m.CC = Var(m.scena, m.t, initialize=0.0, within=NonNegativeReals)
    
    # time derivative of C
    m.dCAdt = DerivativeVar(m.CA, wrt=m.t)  
    m.dCBdt = DerivativeVar(m.CB, wrt=m.t)  
    m.dCCdt = DerivativeVar(m.CC, wrt=m.t)  

    def k1_rule(m,s,t):
        return m.A1[s]*exp(-m.E1[s]*1000/m.R/m.T[t])

    def k2_rule(m,s,t):
        return m.A2[s] * exp(-m.E2[s]*1000/m.R/m.T[t])

    m.kp1 = Var(m.scena, m.t, initialize = k1_rule) # 1/hr
    m.kp2 = Var(m.scena, m.t, initialize = k2_rule)
    
    def cal_kp1(m,z,t):
        '''
        Create the perturbation parameter sets 
        m: model
        z: scenario number
        t: time
        '''
        # LHS: 1/h
        # RHS: 1/h*(kJ/mol *1000J/kJ / (J/mol/K) / K)
        return m.kp1[z,t] == m.A1[z]*exp(-m.E1[z]*1000/(m.R*m.T[t])) 
            
    def cal_kp2(m,z,t):
        '''
        Create the perturbation parameter sets 
        m: model
        z: m.pert, upper or normal or lower perturbation
        t: time
        '''
        # LHS: 1/h
        # RHS: 1/h*(kJ/mol *1000J/kJ / (J/mol/K) / K)
        return m.kp2[z,t] == m.A2[z]*exp(-m.E2[z]*1000/(m.R*m.T[t])) 
    
    # Calculate model response variables
    def CA_conc(m,z,t):
        '''
        Calculate the model predictions
        Argument: 
            z: scenario
            t: timepoints
        '''
        return m.dCAdt[z,t] == -m.kp1[z,t]*m.CA[z,t]
    
    def CB_conc(m,z,t):
        '''
        Calculate the model predictions
        Argument: 
            z: scenario
            t: timepoints
        '''
        return m.dCBdt[z,t] == m.kp1[z,t]*m.CA[z,t] - m.kp2[z,t]*m.CB[z,t]
    
    def CC_conc(m,z,t):
        '''
        Calculate the model predictions
        Argument: 
            z: scenario
            t: timepoints
        '''
        return m.CC[z,t] == m.CA0[0] - m.CA[z,t] - m.CB[z,t]
    
        
    def T_const(m,t):
        if t==0:
            return Constraint.Skip
        else:
            return m.T[t] == m.T[0]
        
        
    # calculating C, Jacobian, FIM
    m.k1_pert_rule = Constraint(m.scena, m.t, rule=cal_kp1)
    m.k2_pert_rule = Constraint(m.scena, m.t, rule=cal_kp2)
    m.dCAdt_rule = Constraint(m.scena, m.t, rule=CA_conc)
    m.dCBdt_rule = Constraint(m.scena, m.t, rule=CB_conc)
    m.dCCdt_rule = Constraint(m.scena, m.t, rule=CC_conc)
    #m.CA0_keep = Constraint(m.t, rule=CA0_const)
    #m.T_keep = Constraint(m.t, rule=T_const)
    

    def obj_rule(m):
        return 0
    
    m.Obj = Objective(rule=obj_rule, sense=maximize)
    
    for z in m.scena:
        m.CB[z,0.0].fix(0.0)
        m.CC[z,0.0].fix(0.0)
    
    for v in variable_name:
        m.A1[0].setlb(theta_p['A1'])
        m.A2[0].setlb(theta_p['A2'])
        m.E1[0].setlb(theta_p['E1'])
        m.E2[0].setlb(theta_p['E2'])
        m.A1[0].setub(theta_p['A1'])
        m.A2[0].setub(theta_p['A2'])
        m.E1[0].setub(theta_p['E1'])
        m.E2[0].setub(theta_p['E2'])
        
    return m

In [8]:
def discretizer(m, NFE=32):
    discretizer = TransformationFactory('dae.collocation')
    discretizer.apply_to(m, nfe=NFE, ncp=3, wrt=m.t)
    #m = discretizer.reduce_collocation_points(m, var=m.T, ncp=1, contset=m.t)
    return m 

In [12]:
scena_version = True
dae = False

sigma_p = np.array([[1, 0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0, 1]])

if scena_version:
    variable_name = ['A1[0]', 'A2[0]', 'E1[0]', 'E2[0]']
    theta_p = {'A1[0]': 84.79085853498033, 'A2[0]': 371.71773413976416, 'E1[0]': 7.777032028026428, 'E2[0]': 15.047135137500822}
    # create the model
    scena_this = {'scena-name': [0], 'A1':84.79085853498033,'A2': 371.71773413976416,
                             'E1':7.777032028026428, 'E2':15.047135137500822}
    
    if dae:
        model_un = create_model_dae(scena_this, CA_init=5, T_init=300)
        model_un = discretizer(model_un)
        model_un.T.fix()
    
    else:
        model_un = create_model_scena(scena_this, CA_init=5, T_init=300)
        #model_un = create_model(scena_this, CA_init=5, T_init=300)
        #model_un = add_comp(model_un)
    
else:
    variable_name = ['A1', 'A2','E1','E2']
    theta_p = {'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}
    model_un = create_model(CA_init=5,  T_init=300)



In [13]:
options = 'get_dsdp'
    
    
# get information
time1 = time.time()

if options=='prop':
    results = propagate_uncertainty(model_un,theta_p, sigma_p, variable_name)
elif options=='get_dsdp':
    dsdp_re, col = get_dsdp(model_un, variable_name, theta_p, tee=True)
elif options=='get_sens':
    gradient_f, gradient_c, line_dic =  get_sensitivity(model_un, variable_name)

#gradient_f, gradient_c, col,row, line_dic = get_dfds_dcds(model_un, variable_name, tee=True)

time2 = time.time()

total_time = time2-time1
print('Total time:', total_time)


W[K_AUG]...	[K_AUG_ASL]No n_rhs declared
W[K_AUG]...	[K_AUG_ASL]Target log10mu:= -11.
W[K_AUG]...	[K_AUG_ASL]This problem has no degrees of freedom
Pass the option square_override for the desired behaviour
W[K_AUG_ASL]...	[K_AUG_ASL]No ipopt_zL_out suffix declared, setting zL = 0.
W[K_AUG_ASL]...	[K_AUG_ASL]No ipopt_zU_out suffix declared, setting zU = 0.
W[K_AUG]...	[K_AUG_ASL]No f_timestamp suffix declared, Fallback to default writing mode.
I[K_AUG]...	[K_AUG_ASL] Filename for dot_sens dsdp_in_.in
W[K_AUG]...	[K_AUG_ASL]dsdp for linear C(x) + I*p = 0 override.
E[K_AUG]...	[SIGMA_COMPUTE]is x[0] = xlb?
E[K_AUG]...	[SIGMA_COMPUTE]is x[0] = xlb?
E[K_AUG]...	[SIGMA_COMPUTE]is x[1] = xlb?
E[K_AUG]...	[SIGMA_COMPUTE]is x[1] = xlb?
E[K_AUG]...	[SIGMA_COMPUTE]is x[2] = xlb?
E[K_AUG]...	[SIGMA_COMPUTE]is x[2] = xlb?
E[K_AUG]...	[SIGMA_COMPUTE]is x[3] = xlb?
E[K_AUG]...	[SIGMA_COMPUTE]is x[3] = xlb?
E[K_AUG]...	[SIGMA_COMPUTE]Unresolved sigma values, is the mu computation successful? or is thi

## IDAES code analysis

In [15]:
if options=='prop':
    var_idx = np.array([True,True,False,False,True])
    dsdp_array = results.dsdp.toarray()
    
elif options=='get_dsdp':
    dsdp_array = dsdp_re.toarray().T
    
    
print(col)
timepoint = [0.125,0.25,0.375,0.5,0.625,0.75,0.875,1.0]
index_list = []

measure_name = []
for cname in ['CA', 'CB', 'CC']:
    for tim in timepoint:
        gene_name = cname+'[0,'+str(tim)+']'
        measure_name.append(gene_name)

for itera in range(len(measure_name)):
    no = col.index(measure_name[itera])
    index_list.append(no)

dsdp_extract = []
for kk in index_list:
    dsdp_extract.append(dsdp_array[kk])
    
print(dsdp_extract)

jac = {}
jac['A1'] = []
jac['A2'] = []
jac['E1'] = []
jac['E2'] = []

for d in range(len(dsdp_extract)):
    jac['A1'].append(dsdp_extract[d][0])
    jac['A2'].append(dsdp_extract[d][1])
    jac['E1'].append(dsdp_extract[d][2])
    jac['E2'].append(dsdp_extract[d][3])
    
print(jac)

#print(dsdp_array)
#print(np.shape(dsdp_array))
dd=pd.DataFrame(dsdp_array)
dd.to_csv('dsdp_test.csv')

['A1[0]', 'A2[0]', 'E1[0]', 'E2[0]', 'kp1[0,0.125]', 'kp1[0,0.25]', 'kp1[0,0.375]', 'kp1[0,0.5]', 'kp1[0,0.625]', 'kp1[0,0.75]', 'kp1[0,0.875]', 'kp1[0,1.0]', 'kp2[0,0.125]', 'kp2[0,0.25]', 'kp2[0,0.375]', 'kp2[0,0.5]', 'kp2[0,0.625]', 'kp2[0,0.75]', 'kp2[0,0.875]', 'kp2[0,1.0]', 'CA[0,0.0]', 'CA[0,0.125]', 'CA[0,0.25]', 'CA[0,0.375]', 'CA[0,0.5]', 'CA[0,0.625]', 'CA[0,0.75]', 'CA[0,0.875]', 'CA[0,1.0]', 'CB[0,0.0]', 'CB[0,0.125]', 'CB[0,0.25]', 'CB[0,0.375]', 'CB[0,0.5]', 'CB[0,0.625]', 'CB[0,0.75]', 'CB[0,0.875]', 'CB[0,1.0]', 'CC[0,0.0]', 'CC[0,0.125]', 'CC[0,0.25]', 'CC[0,0.375]', 'CC[0,0.5]', 'CC[0,0.625]', 'CC[0,0.75]', 'CC[0,0.875]', 'CC[0,1.0]', 'kp1[0,0.0]', 'kp2[0,0.0]']
[array([-0.01730277,  0.        ,  0.58817855,  0.        ]), array([-0.02164992,  0.        ,  0.73595264,  0.        ]), array([-0.02031694,  0.        ,  0.69064014,  0.        ]), array([-0.01694758,  0.        ,  0.57610447,  0.        ]), array([-0.01325343,  0.        ,  0.45052817,  0.        ]), arra

## Get sensitivity

In [ ]:
print(line_dic)
print(np.shape(gradient_c))
print(gradient_c)


In [ ]:
jaco_store = pd.DataFrame({'A1': gradient_c.T[0],
                          'A2': gradient_c.T[1],
                          'E1': gradient_c.T[2],
                          'E2': gradient_c.T[3]})
print(jaco_store)

In [ ]:
k1_ext = value(model_un.k1)
k2_ext = value(model_un.k2)
timepoint = [0.0, 0.125, 0.25, 0.375, 0.5,0.625, 0.75, 0.875, 1.0]
CA = []
CB = []
CC = []
for i in timepoint:
    CA.append(value(model_un.C['CA',i]))
    CB.append(value(model_un.C['CB',i]))
    CC.append(value(model_un.C['CC',i]))
    
dCAdt = []
dCBdt = []
dCCdt = []

for j in range(9):   
    dCAdt.append(-k1_ext*CA[j])
    dCBdt.append(k1_ext*CA[j]-k2_ext*CB[j])
    dCCdt.append(k2_ext*CB[j])
print('dCAdt:', dCAdt)
print('dCBdt:', dCBdt)
print('dCCdt:', dCCdt)